<a href="https://colab.research.google.com/github/JasmineAdvanture/fatty-liver-project/blob/BDS-13-CKD/CKD_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving NTCMRC_all.xlsx to NTCMRC_all.xlsx


In [2]:
#Imports
import pandas as pd
import numpy as np

#read dataset as df
df = pd.read_excel("NTCMRC_all.xlsx")

# Create a copy of df as df1
df1 = df.copy()

# Replace '\\N' with NaN
df1 = df1.replace('\\N', np.nan)

# Specify the columns to be converted to float
columns_to_convert1 = ['BMI', 'Triglyceride_y', 'gamgt', 'waist_y', 'mst', 'egfrn', 'Estimated_GFR_x', 'Alb_Cre_ratio', 'HOMA_IR', 'HS_CRP', \
                       'LDL_C_direct', 'LDL_C_HDL_C', 'Adiponectin', 'Leptin', 'Uric_Acid','Insulin']

# Specify the columns to be converted to int
columns_to_convert2 = ['脂肪肝 fatty Liver (0:正常  1:mild 2:moderate 3:severe)', 'smoke', 'smoke_q']


# Convert the specified columns to float and fill missing/unconvertible values with NaN
for column in columns_to_convert1:
    df1[column] = pd.to_numeric(df1[column], errors='coerce')

# Convert the specified columns to int and fill missing/unconvertible values with NaN
for column in columns_to_convert2:
    df1[column] = pd.to_numeric(df1[column], errors='coerce').astype(float).astype(pd.Int64Dtype())

# Calculate FLI using the formula and defined as df2
df2 = df1.copy()
df2['FLI'] = (np.exp(0.953 * np.log(df2['Triglyceride_y']) + 0.139 * df2['BMI'] + 0.718 * np.log(df2['gamgt']) \
     + 0.053 * df2['waist_y'] - 15.745)) / (1 + np.exp(0.953 * np.log(df2['Triglyceride_y']) \
    + 0.139 * df2['BMI'] + 0.718 * np.log(df2['gamgt']) + 0.053 * df2['waist_y'] - 15.745)) * 100

# Derive FL_echo based on ultrasound results column
df2['FL_echo'] = df2['脂肪肝 fatty Liver (0:正常  1:mild 2:moderate 3:severe)']
df2['FL_echo'] = df2['FL_echo'].replace('<NA>', np.nan)

#Derive fl_status column based on fl_echo and FLI(when available): 1 for positive; 0 for negative; -1 for unavailable
def derive_fl_status(row):
    # Derive FL_Check column to infer the status by echo or FLI
    liver_status = row['脂肪肝 fatty Liver (0:正常  1:mild 2:moderate 3:severe)']
    fli_value = row['FLI']

    if pd.isna(liver_status) and pd.isna(fli_value):
        return -1
    elif pd.notna(liver_status) and liver_status != 0:
        return 1
    elif pd.notna(fli_value) and fli_value >= 60:
        return 1
    else:
        return 0

df2['fl_status'] = df2.apply(derive_fl_status, axis=1)

#Derive homa_ir_check, hs_crp_check, and mst_total to determine MAFLD risk factors
df2['homa_ir_check'] = df2['HOMA_IR'].apply(lambda x: 1 if x >= 2.5 else 0)
df2['hs_crp_check'] = df2['HS_CRP'].apply(lambda x: 1 if x > 2 else 0)
df2['mst_total'] = df2[['w', 'hyper', 'HDL', 'fg', 'trig', 'homa_ir_check', 'hs_crp_check']].sum(axis=1)


#Derive target variables and named as df3
def derive_MAFLD(df):
    df['MAFLD'] = 0  # Initialize MAFLD field as 0

    # Condition 1: fl_status = -1
    df.loc[df['fl_status'] == -1, 'MAFLD'] = -1

    # Condition 2: fl_check = 0
    df.loc[df['fl_status'] == 0, 'MAFLD'] = 0

    # Condition 3: fl_check = 1
    # Subcondition 1: BMI >= 23
    df.loc[(df['fl_status'] == 1) & (df['BMI'] >= 23), 'MAFLD'] = 1

    # Subcondition 2: BMI < 23 and mst >= 2
    df.loc[(df['fl_status'] == 1) & (df['BMI'] < 23) & (df['mst_total'] >= 2), 'MAFLD'] = 1

    # Subcondition 3: DM_determine = 1
    df.loc[(df['fl_status'] == 1) & (df['DM_determine'] == 1), 'MAFLD'] = 1

    return df

def derive_CKD(df):
    # Initialize CKD field as -1
    df['CKD'] = -1

    # Condition 1: egfrn >= 60 and Alb_Cre_ratio < 3
    df.loc[(df['Estimated_GFR_x'] >= 60) & (df['Alb_Cre_ratio'] < 3), 'CKD'] = 1

    # Condition 2: egfrn >= 60 and 3 <= Alb_Cre_ratio <= 30 or 45 <= egfrn < 60 and Alb_Cre_ratio < 3
    df.loc[((df['Estimated_GFR_x'] >= 60) & (df['Alb_Cre_ratio'].between(3, 30))) |
           ((df['Estimated_GFR_x'].between(45, 60)) & (df['Alb_Cre_ratio'] < 3)), 'CKD'] = 2

    # Condition 3: egfrn >= 60 and Alb_Cre_ratio > 30 or egfrn < 60 and Alb_Cre_ratio >= 0
    df.loc[((df['Estimated_GFR_x'] >= 60) & (df['Alb_Cre_ratio'] > 30)) |
           ((df['Estimated_GFR_x'] < 60) & (df['Alb_Cre_ratio'] >= 0)), 'CKD'] = 3

    # Set CKD as 0 for cases where egfrn and Alb_Cre_ratio are not empty and CKD is still -1
    df.loc[(df['Estimated_GFR_x'].notnull()) & (df['Alb_Cre_ratio'].notnull()) & (df['CKD'] == -1), 'CKD'] = 0

    return df

df3 = derive_CKD(derive_MAFLD(df2))

In [3]:
# Derive FL_group_list, 对CMRC_id分组并计算每个病人的FL_Check的唯一值
grouped = df2.groupby('CMRC_id')['fl_status'].unique()
df2['fl_group_list'] = df2.groupby('CMRC_id')['fl_status'].transform(lambda x: [x.unique().tolist()] * len(x))

# Derive patient_fl_validity according to FL_group_list conditions
def assign_patient_fl_validity(df):
    df['patient_fl_validity'] = -1  # 初始化所有记录为第三组 (-1)

    def get_patient_valid(group_list):
        if isinstance(group_list, list):
            if -1 in group_list and len(group_list) == 1:
                return "unavailable"  # 第三组 (-1)
            elif -1 in group_list:
                return "partial"  # 第二组 (0)
            else:
                return "completed"  # 第一组 (1)
        else:
            return "others"  # 第三组 (-1)

    df['patient_fl_validity'] = df['fl_group_list'].apply(get_patient_valid)

    return df


#assign patient valid value
df4 = assign_patient_fl_validity(df3)

In [10]:
# TODO: Need to add to utils for future usage
def sliding_window_data(df, input_window_size, target_window_size, target_var):
    transformed_data = []
    group_counter = {}

    df_sorted = df.sort_values(['CMRC_id', 'year_come'])

    for patient_id, group in df_sorted.groupby('CMRC_id'):
        if len(group) < input_window_size + target_window_size:
            continue

        group_counter.setdefault(patient_id, 0)
        group_counter[patient_id] += 1
        group_alias = f'{patient_id}_group{group_counter[patient_id]}'

        for i in range(len(group) - input_window_size - target_window_size + 1):
            input_data = group[i:i+input_window_size]
            target_data = group[i+input_window_size:i+input_window_size+target_window_size]

            # Flatten input_data and repeat target_data
            input_features_t1 = input_data.iloc[0, :].values.flatten()
            input_features_t2 = input_data.iloc[1, :].values.flatten()
            target_var_values = target_data[target_var].values

            new_row = [group_alias] + list(input_features_t1) + list(input_features_t2) + list(target_var_values)

            transformed_data.append(new_row)

    columns_list = ['CMRC_id'] + [f't1_{col}' for col in input_data.columns] + [f't2_{col}' for col in input_data.columns] + [f't3_{target_var}']
    transformed_df = pd.DataFrame(transformed_data, columns=columns_list)
    return transformed_df

# Sliding window implementation, ie. use previous 2 years record to predict the 3 year MAFLD status
df5 = sliding_window_data(df4, input_window_size=2, target_window_size=1, target_var='CKD')


In [11]:
#checks
df5[['t1_CKD','t2_CKD', 't3_CKD']].value_counts(dropna=False).reset_index()

,t1_CKD,t2_CKD,t3_CKD,0
0,2,2,2,4075
1,3,3,3,788
2,2,2,-1,527
3,2,2,3,375
4,2,-1,2,358
...,...,...,...,...
56,3,1,-1,3
57,3,1,2,2
58,1,3,1,1
59,-1,3,1,1


In [12]:
df5[(df5['t3_CKD'] != -1)][['t1_CKD','t2_CKD', 't3_CKD']].value_counts(dropna=False).reset_index()

,t1_CKD,t2_CKD,t3_CKD,0
0,2,2,2,4075
1,3,3,3,788
2,2,2,3,375
3,2,-1,2,358
4,1,2,2,312
5,2,1,2,252
6,2,3,2,215
7,2,3,3,205
8,-1,2,2,202
9,2,2,1,190


In [13]:
# Filter available data that can be applied to models
# TODO: Need Discussion for filtered/selected data
df6 = df5[(df5['t3_CKD'] != -1)]

# Drop ID relevant cols in the dataset
columns_to_drop = ['CMRC_id', 't1_CMRC_id', 't2_CMRC_id', 't1_sid', 't2_sid', 't1_P_Number','t2_P_Number']
df7 = df6.drop(columns=columns_to_drop)

#Select key columns for conventional machine learning models

# This function is for adding prefix for cols, the cols should be a list of column names that needs to add prefix(such as "t1_" in this project)
# TODO: Add func in utils
def add_prefix(cols, prefixes):
# Note the prefixes should be a LIST, eg. prefixes = ["t1_", "t2_"]
    renamed_columns = []
    for prefix in prefixes:
        renamed_columns.extend([prefix + column for column in cols])
    return renamed_columns

columns = ["sex", "age", "waist_y", "Glucose_AC_y", "Triglyceride_y", "HDL_C_y", "AST_GOT", "ALT_GPT", \
          "gamgt", "Insulin", "T_Cholesterol", "LDL_C_direct", "VLDL_C", "Non_HDL_C", "T_CHOL_HDL_C", \
          "LDL_C_HDL_C", "HS_CRP", "Hb_A1c", "Uric_Acid", "HBsAg_x", "Anti_HCV_x", "HOMA_IR", "Adiponectin", \
           "Leptin", "TotalVitaminD", "smoke", "smoke_q", "coffee", "betel", "BMI", "DM_determine", "w", "hyper", \
           "fg", "HDL", "trig", "sarcf", "ms2", "MNA", "AUDIT", "HBV_", "HCV_", "MAFLD", "CKD"]
prefixes = ["t1_", "t2_"]
renamed_columns = add_prefix(columns, prefixes)

# Add selected features and target var
df8 = df7[renamed_columns]
df8['t3_CKD'] = df7['t3_CKD']


<ipython-input-13-31346697f28f>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df8['t3_CKD'] = df7['t3_CKD']


In [17]:
# Deal with HBsAg_x, Anti_HCV_x, (values eg. 陰性    0.351)
# This function will extract the numeric values
import re
def extract_numeric_value(value):
    pattern = r'\d+(\.\d+)?'  # 正则表达式模式，匹配一个或多个数字（包括小数点）
    match = re.search(pattern, str(value))
    if match:
        return float(match.group())
    else:
        return None

# extract numeric values for these cols and rename that as  *_num
columns_to_extract = ['t1_HBsAg_x', 't2_HBsAg_x', 't1_Anti_HCV_x', 't2_Anti_HCV_x']

df9 = df8.copy()
for column in columns_to_extract:
    new_column_name = column + '_num'
    df9[new_column_name] = df9[column].apply(extract_numeric_value)

# Specify the columns to be converted to float
columns_to_convert1 = ['t1_LDL_C_direct', 't1_LDL_C_HDL_C', 't1_Adiponectin', 't1_Leptin', 't1_Uric_Acid','t1_Insulin', \
                       't2_LDL_C_direct', 't2_LDL_C_HDL_C', 't2_Adiponectin', 't2_Leptin', 't2_Uric_Acid','t2_Insulin',]

# Specify the columns to be converted to int
columns_to_convert2 = ['t1_smoke', 't1_smoke_q', 't2_smoke', 't2_smoke_q']

# Convert the specified columns to float and fill missing/unconvertible values with NaN
for column in columns_to_convert1:
    df9[column] = pd.to_numeric(df9[column], errors='coerce')

# Convert the specified columns to int and fill missing/unconvertible values with NaN
for column in columns_to_convert2:
    df9[column] = pd.to_numeric(df9[column], errors='coerce').astype(float).astype(pd.Int64Dtype())

# drop these cols as those been derived for numeric cols, alias *_num
cols_to_drop = ['t1_HBsAg_x', 't2_HBsAg_x', 't1_Anti_HCV_x', 't2_Anti_HCV_x']
df9 = df9.drop(cols_to_drop, axis=1)

In [18]:
# check stats
df9['t3_CKD'].value_counts(dropna=False)

2    6026
3    1615
1     627
Name: t3_CKD, dtype: int64

Conventional Model Implementation

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import mean_squared_error, confusion_matrix, precision_score, accuracy_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# features and target variables setup
features = df9.columns.drop(['t3_CKD'])
target = 't3_CKD'
X = df9[features]
y = df9[target]

#Deal with missing values
# TODO, need discussion
imputer = SimpleImputer(strategy='median')
X = imputer.fit_transform(X)

# Resolve the log model cannot converge issue
# Standarize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# TODO Discussion for CKD=0 status - now will map to 0, 1, 2 classes
# from sklearn.preprocessing import LabelEncoder
# label_encoder = LabelEncoder()
# y = label_encoder.fit_transform(y)
# y = y.map({'1': 0, '2': 1, '3': 2})

# train test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=101)

In [35]:
from sklearn.metrics import roc_auc_score

# Logistic Regression model
logmodel = LogisticRegression(max_iter=2000, multi_class='multinomial')
logmodel.fit(X_train, y_train)

# Evaluation
predictions_log = logmodel.predict(X_test)
probabilities_log = logmodel.predict_proba(X_test)

print(classification_report(y_test, predictions_log))

# Calculate AUC for each class
auc_scores = []
for i in range(logmodel.classes_.shape[0]):
    auc = roc_auc_score((y_test == logmodel.classes_[i]).astype(int), probabilities_log[:, i])
    auc_scores.append(auc)

print("AUC (Logistic Regression):", auc_scores)
# Output each AUC
for i, auc in enumerate(auc_scores):
    class_label = logmodel.classes_[i]
    print(f"Log Model AUC (Class {class_label}): {auc}")

              precision    recall  f1-score   support

           1       0.30      0.07      0.12       202
           2       0.81      0.95      0.88      1798
           3       0.83      0.56      0.67       481

    accuracy                           0.80      2481
   macro avg       0.65      0.53      0.56      2481
weighted avg       0.77      0.80      0.78      2481

AUC (Logistic Regression): [0.7919771134638695, 0.7582404070245612, 0.863383575883576]
Log Model AUC (Class 1): 0.7919771134638695
Log Model AUC (Class 2): 0.7582404070245612
Log Model AUC (Class 3): 0.863383575883576


In [34]:
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score

# SVM Model
svm_model = SVC(probability=True)

# train and predictions
svm_model.fit(X_train, y_train)
predictions_svm = svm_model.predict(X_test)
probabilities = svm_model.predict_proba(X_test)

print(classification_report(y_test, predictions_svm))
# Calculate AUC for each class
auc_scores = []
for i in range(svm_model.classes_.shape[0]):
    y_true = (y_test == svm_model.classes_[i]).astype(int)
    y_score = probabilities[:, i]
    auc = roc_auc_score(y_true, y_score)
    auc_scores.append(auc)

# Output each AUC
for i, auc in enumerate(auc_scores):
    class_label = svm_model.classes_[i]
    print(f"SVM Model AUC (Class {class_label}): {auc}")

              precision    recall  f1-score   support

           1       0.50      0.00      0.01       202
           2       0.80      0.97      0.88      1798
           3       0.85      0.51      0.64       481

    accuracy                           0.81      2481
   macro avg       0.72      0.50      0.51      2481
weighted avg       0.79      0.81      0.76      2481

SVM Model AUC (Class 1): 0.7623110709491309
SVM Model AUC (Class 2): 0.7543594069871029
SVM Model AUC (Class 3): 0.8633721413721416
